In [1]:
import sys
sys.executable

'/home/goonieb/gitrep/bandstour/venv/bin/python'

In [21]:
import os
import json
import pymongo
import musicbrainzngs
import arrow
import re
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict, OrderedDict

from dateutil import parser
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [22]:
###BEFORE RUNNING THIS ONE YOU SHOULD CLEAN YOUR MONGO DB OF artist strings starting with &, with + and such, or remove the JSon files first"
#example :db.getCollection('selectedArtists').find({"_id": /&.*/})
#this needs to be done by hand

##Variables
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
# setup mongo
client = MongoClient()
db = client["bandstour"]
venues =  db["venues"]
artists =  db["selectedArtists"]
FUZZ_RATIO = 90
NB_remplacements = 0
##input function
##https://gist.github.com/hrouault/1358474
def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.
    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).
    The "answer" return value is one of "yes" or "no".
    """
    valid = {"yes":True,   "y":True,  "ye":True,
             "no":False,     "n":False}
    if default == None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = raw_input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "\
"(or 'y' or 'n').\n")
#Clean a string with regex
def slugify(s):
    """
    Simplifies ugly strings into something URL-friendly.
    >>> print slugify("[Some] _ Article's Title--")
    some-articles-title
    """

    s = re.sub('\s+', ' ', s)
    s = s.replace(' ', '-')

    s = re.sub('\W+', ' ', s)

    s = s.strip()

    s = s.lstrip()
    s = re.sub('\W+', '_', s)
    return s

#WE WANT TO LOOP OVER ARTISTS IN ORDER TO DEDUPLICATE THE RECORDS AND ASSIGN TOURS TO ONE ENTITY ONLY.
#ex: "   dede" =add his dates to> "dede" who already has his.
#in case there is an ambiguity, we will have to choose, at the end
replacement_list =[]
print "  _____________________________"
print " |_couples_d'ARTISTES_de_dups__|"
print " |_NOM_|_SLUG_|nb_dates_|_mbid_|"

  _____________________________
 |_couples_d'ARTISTES_de_dups__|
 |_NOM_|_SLUG_|nb_dates_|_mbid_|


In [10]:
for artistA in artists.find({},no_cursor_timeout=True) :
            #print "AAAA  =>", artistA["_id"]
            #print "AAAA slug  =>",slugify(artistA["_id"]).lower()
            
            for artistB in artists.find({},no_cursor_timeout=True) :
                if artistA["_id"]==artistB["_id"]:
                    pass
                elif (fuzz.ratio(\
                                        slugify(artistA["_id"]).lower()\
                                        , slugify(artistB["_id"]).lower()\
                                       ) > FUZZ_RATIO and len(artistA["gigs"]) >= len(artistB["gigs"])) :
                    print "A|", artistA["_id"],"|",slugify(artistA["_id"]).lower(),"|",len(artistA["gigs"]),"|",artistA["mbid"],"|"
                    print "B|", artistB["_id"],"|",slugify(artistB["_id"]).lower(),"|",len(artistB["gigs"]),"|",artistB["mbid"],"|"
                    print "_______________________________"
                    for gig in artistB["gigs"]:
                        artistA["gigs"].append(gig)
                        
                    Aprime = artistA["gigs"]
                    #print Aprime
                    Aprime2 = sorted(Aprime,key=lambda t: t["datetime"].strftime(DATETIME_FORMAT))
                    #print "Aprime2",Aprime2
                    #BYPASS THE FOLLOWING LINE  WHEN THE SCRIPT IS MATURE
                    #Answer_art= query_yes_no("MERGE "+artistB["_id"]+ " WITH "+ artistA["_id"]+" ?" , default="no")
                    #print Answer_art
                    Answer_art = True
                    if Answer_art == True :
                        artistA["gigs"]=Aprime2
                        #print artistA["gigs"]
                        replacement_list.append([artistA["_id"],artistB["_id"],artistA])
                        
print "DONE!"
for  replacement in replacement_list:
    print replacement
    alias= replacement[1] 
    replacement[2]["alias"].append(alias)
    #artists.update({"_id" :replacement[0]}, replacement[2], upsert=True, multi=False)
    #artists.remove({"_id" :replacement[1]})

KeyboardInterrupt: 

In [183]:
from slugify import slugify
CSVDATA=[]
res = artists.find({})
print "NB OCCURENCES:",res.explain()['executionStats']["nReturned"]
COUNTER=0
for rep in res:
    COUNTER+=1
    a=str(rep.get('_id').encode('utf-8'))
    #print a
    CSVDATA.append([str(COUNTER),slugify(a.lower()),a])
        #print re['_id']
        #print CSVDATA 
    #    if len(CSVDATA>1:
    #           if (fuzz.ratio(re['_id'].lower(), CSVDATA[len(CSVDATA)]) >= 30) :
    #                print re['_id']
    #                CSVDATA.append(["\""+re['_id']+"\""])
    #        #CSVDATA.append([re['_id']])
     
print COUNTER
print CSVDATA[:5]

NB OCCURENCES: 179054
179054
[['1', 'amy-kaiser-director', '       Amy Kaiser, director'], ['2', 'kevin-mcbeth-director', '       Kevin McBeth, director'], ['3', 'carnovoure-breat', '    Carnovoure Breat'], ['4', 'waveradio', '   WaveRadio'], ['5', 'blackened-metallica-tribute', '  Blackened (Metallica Tribute)     ']]


In [184]:
from operator import itemgetter 
CSVDATA2=sorted(CSVDATA ,key =itemgetter(1)) 
print CSVDATA2[20:120]

[['368', '1-8-7', '1.8.7'], ['361', '1-800-dinosaur', '1-800 DINOSAUR'], ['346', '1-amazing-show-think-pink-floyd', '1 AMAZING SHOW-THINK PINK FLOYD'], ['347', '1-amvrka', '1 AMVRKA'], ['348', '1-beat-off', '1 Beat Off'], ['349', '1-big-mob', '1 Big Mob'], ['211', '1-dads', '#1 Dads'], ['362', '1-drop-nation', '1-Drop Nation'], ['350', '1-echo-1', '1 Echo 1'], ['351', '1-girl-nation', '1 Girl Nation'], ['352', '1-hour-crew', '1 Hour Crew'], ['356', '1-in-five', '1 in Five'], ['212', '1-kid', '#1 Kid'], ['353', '1-last-chance', '1 Last Chance'], ['363', '1-o-a-k', '1-O.A.K.'], ['357', '1-oz-jig', '1 oz. Jig'], ['354', '1-set-radiohead-1-set-original', '1 Set Radiohead + 1 Set Original'], ['364', '1-up-creative-mind-frame', '1-UP Creative Mind Frame'], ['365', '1-up-creative-mindframe', '1-Up Creative Mindframe'], ['355', '1-year-brutal-and-sadistic-show', '1 Year Brutal and Sadistic Show'], ['369', '10', '10'], ['370', '10-000-russos', '10 000 Russos'], ['426', '10-00am-venice-symphony-

In [193]:
rowOLD=None
CNT=0
liste_doublons=[]
for row in CSVDATA2:
    #print row[1]
    if  rowOLD is not None and (fuzz.ratio(row[1], rowOLD[1]) >= 95) :
        print row ,rowOLD
        
        liste_doublons.append([row ,rowOLD])
        rowOLD=row
        CNT+=1
        
    if rowOLD is  None:
        
        rowOLD=row
    else:
        rowOLD=row
print len(CSVDATA2)
print CNT
print liste_doublons


['365', '1-up-creative-mindframe', '1-Up Creative Mindframe'] ['364', '1-up-creative-mind-frame', '1-UP Creative Mind Frame']
['377', '10-string-symphony', '10 String Symphony'] ['376', '10-sting-symphony', '10 Sting Symphony']
['573', '1st-natures', '1st Natures'] ['572', '1st-nature', '1st Nature']
['707', '25th-annniversary-tour', '25th annniversary tour'] ['706', '25th-anniversary-tour', '25th anniversary tour']
['824', '3-parts-dead', '3 Parts Dead'] ['823', '3-part-dead', '3 Part Dead']
['890', '36-crazyfists', '36 Crazyfists'] ['889', '36-crayfists', '36 Crayfists']
['892', '36-crazyfists-skinlab-and-all-hail-the-yeti', '36 Crazyfists, Skinlab and All Hail the Yeti'] ['891', '36-crazyfists-skinlab-all-hail-the-yeti', '36 Crazyfists, Skinlab & All Hail the Yeti']
['949', '3rd-times-a-charm', '3rd Times a Charm'] ['948', '3rd-time-s-a-charm', "3rd Time's a Charm"]
['957', '4-asprin-morning', '4 Asprin Morning '] ['956', '4-aspirin-morning', '4 Aspirin Morning']
['1105', '50ftwoman

['32502', 'creative-adults', 'Creative Adults'] ['32501', 'creative-adult', 'Creative Adult']
['32515', 'creature-comforts', 'Creature Comforts'] ['32514', 'creature-comfort', 'Creature Comfort']
['32562', 'creepsville666', 'Creepsville666'] ['32561', 'creepsville-666', 'Creepsville 666']
['32624', 'crime-in-stero', 'Crime In Stero'] ['32625', 'crime-in-stereo', 'Crime in Stereo']
['23964', 'crossfader', 'CROSSFADER'] ['32853', 'crossfade', 'Crossfade']
['33027', 'crumbsntchers', 'Crumbsntchers'] ['33026', 'crumbsnatchers', 'CrumbSnatchers']
['175210', 'crushingcrayons', 'crushingcrayons'] ['33064', 'crushing-crayons', 'Crushing Crayons']
['33124', 'cryptopsy', 'Cryptopsy'] ['33123', 'cryptopsty', 'Cryptopsty']
['33246', 'culoe-e-song', 'Culoe e Song'] ['33245', 'culoe-de-song', 'Culoe de Song']
['33361', 'curly-castro', 'Curly Castro'] ['33360', 'curly-casto', 'Curly Casto']
['33503', 'cutty-flam', 'Cutty Flam'] ['33502', 'cutty-fam', 'Cutty Fam']
['33592', 'cypress-hilll', 'Cypress H

['64535', 'hayden-summerall', 'Hayden Summerall'] ['64537', 'hayden-sumerall', 'Hayden sumerall']
['64539', 'hayes-carll', 'Hayes Carll'] ['64538', 'hayes-carl', 'Hayes Carl']
['64614', 'hb-surroundsound', 'Hb Surroundsound'] ['62958', 'hb-surround-sound', 'HB Surround Sound']
['64677', 'headbangers-balls', 'Headbangers Balls'] ['64676', 'headbangers-ball', 'Headbangers Ball']
['64694', 'headless-horsman', 'Headless Horsman'] ['64693', 'headless-horseman', 'Headless Horseman']
['64805', 'heartracers', 'HeartRacers'] ['64845', 'heartracer', 'Heartracer']
['64868', 'heartthrrob', 'Heartthrrob'] ['64867', 'heartthrob', 'Heartthrob']
['64980', 'heaven-shall-burn-unearth-rise-to-remain-neaera', 'Heaven Shall Burn, Unearth, Rise to Remain, Neaera'] ['64979', 'heaven-shall-burn-unearth-rise-to-remain-naeara', 'Heaven Shall Burn, Unearth, Rise to Remain, Naeara']
['65069', 'heavyweight-djs', 'HeavyWeight DJs'] ['65080', 'heavyweight-dj-s', "Heavyweight DJ's"]
['65226', 'helado-negro', 'Helado 

['95560', 'manu-codjia', 'Manu Codjia'] ['95559', 'manu-codja', 'Manu Codja']
['95568', 'manu-franchie', 'Manu Franchie'] ['95567', 'manu-franchi', 'Manu Franchi']
['95604', 'manuel-de-la-mare', 'Manuel de la Mare'] ['95582', 'manuel-de-la-mar', 'Manuel De La Mar']
['95608', 'manufactured-superstars', 'Manufactured Superstars'] ['95607', 'manufacture-superstars', 'Manufacture Superstars']
['95771', 'marc-peilon', 'Marc Peilon'] ['95770', 'marc-peillon', 'Marc Peillon']
['95837', 'marcel-dettmann', 'Marcel Dettmann'] ['95836', 'marcel-dettman', 'Marcel Dettman']
['93246', 'marco-farone', 'MARCO FARONE'] ['95923', 'marco-faraone', 'Marco Faraone']
['96104', 'margot-mcdonald', 'Margot McDonald'] ['96103', 'margot-macdonald', 'Margot MacDonald']
['96114', 'maria-carrelli', 'Maria Carrelli'] ['96113', 'maria-carelli', 'Maria Carelli']
['96121', 'maria-gabrielle-band', 'Maria Gabrielle Band'] ['96120', 'maria-gabriella-band', 'Maria Gabriella Band']
['93256', 'marie-priux', 'MARIE PRIUX'] ['

['124175', 'richard-ellliot', 'Richard Ellliot'] ['124174', 'richard-elliot', 'Richard Elliot']
['124177', 'richard-fraoli', 'Richard Fraoli'] ['124176', 'richard-fraioli', 'Richard Fraioli']
['124193', 'richard-johnston', 'Richard Johnston'] ['124192', 'richard-johnson', 'Richard Johnson']
['124208', 'richard-manetti', 'Richard Manetti'] ['124207', 'richard-manett', 'Richard Manett']
['124373', 'rick-margitza-sax-peter-giron-double-bass-jeff-boudreaux-drums-manuel-rocheman-piano', 'Rick Margitza, sax - Peter Giron, double bass - Jeff Boudreaux, drums - Manuel Rocheman, piano'] ['124372', 'rick-margitza-sax-peter-giron-bass-jeff-boudreaux-drums-manuel-rocheman-piano', 'Rick Margitza, sax - Peter Giron, bass - Jeff Boudreaux, drums - Manuel Rocheman, piano']
['124387', 'rick-rosato', 'Rick Rosato'] ['177153', 'rick-rosatao', 'rick rosatao']
['124455', 'ricky-skags-kentucky-thunder', 'Ricky Skags & Kentucky Thunder'] ['124454', 'ricky-skaggs-kentucky-thunder', 'Ricky Skaggs & Kentucky Th

['151478', 'the-greeting-committee', 'The Greeting Committee'] ['151477', 'the-greeting-comittee', 'The Greeting Comittee']
['151487', 'the-greyboy-allstars', 'The Greyboy Allstars'] ['151486', 'the-greyboy-all-stars', 'The Greyboy All Stars']
['151513', 'the-grizzled-mighy', 'The Grizzled Mighy'] ['151512', 'the-grizzled-mighty', 'The Grizzled Mighty']
['177814', 'the-hand-of-gravilo', 'the Hand of Gravilo'] ['151616', 'the-hand-of-gavrilo', 'The Hand of Gavrilo']
['151671', 'the-hard-ways', 'The Hard ways'] ['151670', 'the-hard-way', 'The Hard Way']
['151712', 'the-hates', 'The Hates'] ['151711', 'the-haters', 'The Haters']
['151720', 'the-haverbrook-disaster', 'The Haverbrook Disaster'] ['151719', 'the-haverbook-disaster', 'The Haverbook Disaster']
['151721', 'the-haverbrook-distaster', 'The Haverbrook Distaster'] ['151720', 'the-haverbrook-disaster', 'The Haverbrook Disaster']
['151751', 'the-headlines', 'The Headlines'] ['151750', 'the-headliners', 'The Headliners']
['152070', 'th

In [194]:
print liste_doublons[:5]
import csv
 
 
myFile = open('doublons_artistes_possibles.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(liste_doublons)
     
print("Writing complete")


[[['365', '1-up-creative-mindframe', '1-Up Creative Mindframe'], ['364', '1-up-creative-mind-frame', '1-UP Creative Mind Frame']], [['377', '10-string-symphony', '10 String Symphony'], ['376', '10-sting-symphony', '10 Sting Symphony']], [['573', '1st-natures', '1st Natures'], ['572', '1st-nature', '1st Nature']], [['707', '25th-annniversary-tour', '25th annniversary tour'], ['706', '25th-anniversary-tour', '25th anniversary tour']], [['824', '3-parts-dead', '3 Parts Dead'], ['823', '3-part-dead', '3 Part Dead']]]
Writing complete


In [227]:
import csv
 
liste_doublonsTRIEE = []
with open('doublons_artistes_possiblesTRIES.csv') as File:
    reader = csv.DictReader(File)
    for row in reader:
        #print row
        
        aa=eval(row["ARTISTA"].replace('\"', ''))
        bb=eval(row["ARTISTB"].replace('\"', ''))
        liste_doublonsTRIEE.append([aa,bb])
        print liste_doublonsTRIEE[:3]

[[['100091', 'michael-bernard-fitzgerald', 'Michael Bernard Fitzgerald'], ['100090', 'michael-bernard-fitzgeral', 'Michael Bernard Fitzgeral']]]
[[['100091', 'michael-bernard-fitzgerald', 'Michael Bernard Fitzgerald'], ['100090', 'michael-bernard-fitzgeral', 'Michael Bernard Fitzgeral']], [['10011', 'arlo-mckinney-the-lonesome-sound', 'Arlo McKinney & The Lonesome Sound'], ['10010', 'arlo-mckinley-the-lonesome-sound', 'Arlo McKinley & The Lonesome Sound']]]
[[['100091', 'michael-bernard-fitzgerald', 'Michael Bernard Fitzgerald'], ['100090', 'michael-bernard-fitzgeral', 'Michael Bernard Fitzgeral']], [['10011', 'arlo-mckinney-the-lonesome-sound', 'Arlo McKinney & The Lonesome Sound'], ['10010', 'arlo-mckinley-the-lonesome-sound', 'Arlo McKinley & The Lonesome Sound']], [['100254', 'michael-janisch-s-paradigm-shift', "Michael Janisch's Paradigm Shift"], ['100253', 'michael-janisch-paradigm-shift', 'Michael Janisch Paradigm Shift']]]
[[['100091', 'michael-bernard-fitzgerald', 'Michael Ber

SyntaxError: invalid syntax (<string>, line 1)

In [233]:

       
for  paire in liste_doublonsTRIEE:
    print paire
    #print paire[0][2]
    winner=None
    looser=None
    for artistA in artists.find({"_id":paire[0][2]}):
        print artistA["mbid"]
        for artistB in artists.find({"_id":paire[1][2]}):
            pass
        #   print artistB["mbid"]
    
            
    if artistA["mbid"] is None and artistB["mbid"] is None :
        #check  gigs count return winner

        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    
        
    elif artistA["mbid"] is None and artistB["mbid"] is not None :
        #ARTISTB WINS
        winner=artistB
        looser=artistA
    elif artistA["mbid"] is not None and artistB["mbid"] is  None:
        #ARTIST A WINS
        winner=artistA
        looser=artistB
    elif artistA["mbid"] is not None and artistB["mbid"] is  not None:
         #check  gigs count nd return winner
        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    #apply modifications
    print "winner is", winner["_id"]
    print "looser is", looser["_id"]
    
    #for gig in looser["gigs"]:
     #   print gig
       # for venue in venues.find({"name": gig["venue"]["name"]  }):
        #    print venue
            
    #print winner["gigs"][0]
    for gig in looser["gigs"]:
        
        winner["gigs"].append(gig)
    print "========"
    #artists.update({"_id" :winner["_id"]}, winner, upsert=True, multi=False)
    
    #artists.remove({"_id" :looser["_id"]})
    print winner["gigs"]

[['100091', 'michael-bernard-fitzgerald', 'Michael Bernard Fitzgerald'], ['100090', 'michael-bernard-fitzgeral', 'Michael Bernard Fitzgeral']]
f1b2f9a6-313b-4543-88de-1bd29f3045ed
winner is Michael Bernard Fitzgerald
looser is Mimi Fox - guitar
[{u'venue': {u'city': u'Calgary', u'name': u'Broken City', u'country': u'Canada', u'region': u'AB', u'longitude': -114.074555, u'place': u'Broken City', u'latitude': 51.042684}, u'datetime': datetime.datetime(2007, 9, 28, 0, 0)}, {u'venue': {u'city': u'Calgary', u'name': u'Ship & Anchor', u'country': u'Canada', u'region': u'AB', u'longitude': -114.072547, u'place': u'Ship & Anchor', u'latitude': 51.037832}, u'datetime': datetime.datetime(2008, 8, 20, 19, 0)}, {u'venue': {u'city': u'Calgary', u'name': u'Hifi', u'country': u'Canada', u'region': u'AB', u'longitude': -114.063014, u'place': u'Hifi', u'latitude': 51.04522}, u'datetime': datetime.datetime(2008, 9, 13, 19, 0)}, {u'venue': {u'city': u'Vancouver', u'name': u'Media Club', u'country': u'Uni

In [45]:
#FOR MANUAL EXTRAS

import csv
import codecs


liste_doublonsTRIEE = []


with open('doublons_artistes_possiblesTRIES2.csv') as File:
    reader = csv.DictReader(File)
    for row in reader:
        #print row
        
        aa=row["ARTISTA"]
        bb=row["ARTISTB"]
        liste_doublonsTRIEE.append([aa,bb])
    print liste_doublonsTRIEE

[['Rend Collective Experiment', 'Rend Collective'], ['Randy Rogers ', 'Randy Rogers Band'], ['Ryan Montbleau Band', 'Ryan Montbleau '], ['Nick Curly', 'Nick Curly (official)'], ['Nick Cave & The Bad Seeds', 'Nick Cave'], ['Netsky (live)', 'Netsky '], ['Napalm Death', 'Napalm Death Uk'], ['Mike Stern Band', 'Mike Stern'], ['Michael Franti & Spearhead', 'Michael Franti'], ['Melvin Seals and JGB', 'Melvin Seals'], ['Lukas Nelson & Promise of the Real', 'Lukas Nelson'], ['Lee Fields', 'Lee Fields & The Expressions'], ['Kurt Elling, voice', 'Kurt Elling'], ['George Thorogood', 'George Thorogood & The Destroyers'], ['G. Love & Special Sauce', 'G. Love'], ['Fitz', 'Fitz & The Tantrums'], ['UB40', 'UB40 Featuring Ali, Astro, And Mickey'], ['Tiger & Woods', 'Tiger and Woods'], ['Sunnery James', 'Sunnery James & Ryan Marciano']]


# -*- coding: utf-8 -*-
       
for  paire in liste_doublonsTRIEE:
    print paire[0]
    print paire[1]
    

In [47]:

       
for  paire in liste_doublonsTRIEE:
    print paire
    #print paire[0][2]
    winner=None
    looser=None
    for artistA in artists.find({"_id":paire[0]}):
        print artistA["mbid"]
        for artistB in artists.find({"_id":paire[1]}):
            pass
        #   print artistB["mbid"]
    
            
    if artistA["mbid"] is None and artistB["mbid"] is None :
        #check  gigs count return winner

        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    
        
    elif artistA["mbid"] is None and artistB["mbid"] is not None :
        #ARTISTB WINS
        winner=artistB
        looser=artistA
    elif artistA["mbid"] is not None and artistB["mbid"] is  None:
        #ARTIST A WINS
        winner=artistA
        looser=artistB
    elif artistA["mbid"] is not None and artistB["mbid"] is  not None:
         #check  gigs count nd return winner
        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    #apply modifications
    print "winner is", winner["_id"]
    print "looser is", looser["_id"]
    
    #for gig in looser["gigs"]:
     #   print gig
       # for venue in venues.find({"name": gig["venue"]["name"]  }):
        #    print venue
            
    #print winner["gigs"][0]
    for gig in looser["gigs"]:
        
        winner["gigs"].append(gig)
    print "========"
    artists.update({"_id" :winner["_id"]}, winner, upsert=True, multi=False)
    
    artists.remove({"_id" :looser["_id"]})
    #print winner["gigs"]

['Rend Collective Experiment', 'Rend Collective']
None
winner is Rend Collective Experiment
looser is Rend Collective
['Randy Rogers ', 'Randy Rogers Band']
winner is Rend Collective Experiment
looser is Rend Collective
['Ryan Montbleau Band', 'Ryan Montbleau ']
a83a16b5-a2a2-4645-9fcc-3e228b09631e
winner is Ryan Montbleau Band
looser is Rend Collective
['Nick Curly', 'Nick Curly (official)']
ca33090a-4750-4f6b-adc0-9047fbcaa5bf
winner is Nick Curly
looser is Nick Curly (official)
['Nick Cave & The Bad Seeds', 'Nick Cave']
172e1f1a-504d-4488-b053-6344ba63e6d0
winner is Nick Cave & The Bad Seeds
looser is Nick Cave
['Netsky (live)', 'Netsky ']
None
winner is Nick Cave
looser is Netsky (live)
['Napalm Death', 'Napalm Death Uk']
ce7bba8b-026b-4aa6-bddb-f98ed6d595e4
winner is Napalm Death
looser is Napalm Death Uk


/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:62: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


['Mike Stern Band', 'Mike Stern']
None
winner is Mike Stern
looser is Mike Stern Band
['Michael Franti & Spearhead', 'Michael Franti']
e23eac42-f904-47da-8599-823ff431bea4
winner is Michael Franti & Spearhead
looser is Michael Franti
['Melvin Seals and JGB', 'Melvin Seals']
None
winner is Melvin Seals
looser is Melvin Seals and JGB
['Lukas Nelson & Promise of the Real', 'Lukas Nelson']
None
winner is Lukas Nelson & Promise of the Real
looser is Lukas Nelson
['Lee Fields', 'Lee Fields & The Expressions']
21a6802c-9700-4ab1-be86-ce9d225ace43
winner is Lee Fields
looser is Lee Fields & The Expressions
['Kurt Elling, voice', 'Kurt Elling']
None
winner is Kurt Elling
looser is Kurt Elling, voice
['George Thorogood', 'George Thorogood & The Destroyers']
444e7c50-7f72-4954-b61c-1717332a27d1
winner is George Thorogood
looser is George Thorogood & The Destroyers
['G. Love & Special Sauce', 'G. Love']
dbf7c761-e332-467b-b4d9-aafe06bbcf8f
winner is G. Love & Special Sauce
looser is G. Love
['Fitz

In [ ]:
  #TIESTO /INGROSSO FIX


In [57]:
winner=None
looser=None
for artistA in artists.find({"_id":"Axwell"}):
        print artistA["mbid"]
        for artistB in artists.find({"_id":"Axwell Λ Ingrosso"}):
            pass
        #   print artistB["mbid"]
    
            
if artistA["mbid"] is None and artistB["mbid"] is None :
        #check  gigs count return winner

        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    
        
elif artistA["mbid"] is None and artistB["mbid"] is not None :
        #ARTISTB WINS
        winner=artistB
        looser=artistA
elif artistA["mbid"] is not None and artistB["mbid"] is  None:
        #ARTIST A WINS
        winner=artistA
        looser=artistB
elif artistA["mbid"] is not None and artistB["mbid"] is  not None:
         #check  gigs count nd return winner
        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    #apply modifications
print "winner is", winner["_id"]
print "looser is", looser["_id"]
    
    #for gig in looser["gigs"]:
     #   print gig
       # for venue in venues.find({"name": gig["venue"]["name"]  }):
        #    print venue
            
    #print winner["gigs"][0]
for gig in looser["gigs"]:
        
        winner["gigs"].append(gig)
print "========"
artists.update({"_id" :winner["_id"]}, winner, upsert=True, multi=False)
    
artists.remove({"_id" :looser["_id"]})
    #print winner["gigs"]

4539050e-e355-4c39-bdfa-e74cb67ce365
winner is Axwell
looser is Axwell Λ Ingrosso


/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:57: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


{u'n': 1, u'ok': 1.0}

In [ ]:
"""" FOUND AT LHE LAST MINUTE WHILE RUNNING NODE/EDGE CREATOR
Rend Collective Experiment
Rend Collective 

Randy Rogers 
Randy Rogers Band

Ryan Montbleau Band
Ryan Montbleau 

Nick Curly
Nick Curly (official)

Axwell
Axwell Λ Ingrosso


Nick Cave & The Bad Seeds
Nick Cave

Netsky (live)
Netsky 

Napalm Death
Napalm Death Uk

Mike Stern Band
Mike Stern

Michael Franti & Spearhead
Michael Franti

Melvin Seals and JGB
Melvin Seals

Lukas Nelson & Promise of the Real
Lukas Nelson

Lee Fields
Lee Fields & The Expressions

Kurt Elling, voice
Kurt Elling

George Thorogood
George Thorogood & The Destroyers

G. Love & Special Sauce
G. Love

Fitz
Fitz & The Tantrums

UB40
UB40 Featuring Ali, Astro, And Mickey

Tiësto
Tiëstolive

Tiger & Woods
Tiger and Woods

Sunnery James
Sunnery James & Ryan Marciano


